In [1]:
!pip install -q transformers
!pip install -q --upgrade datasets
!pip install -q rouge_score
!pip install -q sentencepiece
!pip install -q torchviz
!pip install -q cloud-tpu-client
!pip install -q torch-xla
!pip install -q transformers[torch]
!pip install -q evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 2.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydrive2 1.6.3 requires google-api-python-client>=1.12.5, but you have google-api-python-client 1.8.0 which is incompatible.
earthengine-api 0.1.381 requires google-api-python-client>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatib

In [2]:
import os
import time
from google.colab import drive
from google.colab import files
import pandas as pd
import numpy as np
import random
from collections import defaultdict
from collections import Counter
from wordcloud import WordCloud

from datasets import load_dataset
from datasets import Dataset, DatasetDict
import nltk
nltk.download('punkt')
import torch
from torch.utils.data import Dataset, DataLoader
from torchviz import make_dot

from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers.utils import logging


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
drive.mount('/content/drive', force_remount=True)
#! cd drive
os.chdir("/content/drive/MyDrive/Data 266 Project")
#!ls

Mounted at /content/drive


In [4]:
dataset_gen = DatasetDict.load_from_disk('dataset_small_pred')

In [5]:
dataset_gen

DatasetDict({
    funny: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base', 'generated_bart_base', 'generated_pegasus_base', 'generated_t5_finetuned', 'generated_bart_finetuned'],
        num_rows: 500
    })
    worldnews: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base', 'generated_bart_base', 'generated_pegasus_base', 'generated_t5_finetuned', 'generated_bart_finetuned'],
        num_rows: 500
    })
    askreddit: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base', 'generated_bart_base', 'generated_pegasus_base', 'generated_t5_finetuned', 'generated_bart_finetuned'],
        num_rows: 500
    })
})

In [10]:
dataset_gen['funny'][100]

{'subreddit': 'funny',
 'id': 'cdav71j',
 'content': "Here's something interesting about the 'right and wrong' way to put a toilet paper roll on, as a cat owner:\nPut it the right way, and my cats almost always eventually unroll it to play with it, effectively ruining entire rolls of toilet paper.\nPut it the wrong way, and they leave it alone because its harder to unravel.\nThe right way is the 'right way' for a reason: it makes it very easy to unravel... which is great for humans.... but sometimes, it makes it too easy, for cats.\nSo now at my house, we put our toilet paper on 'the wrong way'. It took some getting used to, and I still find myself wanting to switch it back. But its better than coming home to piles of toilet paper in the bathroom every day.",
 'summary': "Sometimes people have legit reasons for putting it the 'wrong' way.",
 'generated_t5_base': "the right way is the 'right way' for a reason: it makes it very easy to unravel . it took some getting used to, and I still 